In [1]:
import pandas as pd
import csv
from tqdm.autonotebook import tqdm
tqdm.pandas()

/tmp/ipykernel_55214/3420178448.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
pd.set_option('display.max_colwidth', None)

df = pd.read_csv('cpv/all.csv', sep=',', quotechar='"', quoting=csv.QUOTE_MINIMAL, usecols=[1, 2, 3], dtype={'cpv': object})
df.head()

,desc,lang,cpv
0,Marché de travaux d'entretien en plomberie des parties communes et des logements habites des cites de 13 habitat.,FR,45330000
1,Mantenimiento base de datos de instalaciones (BDI) y Scada.,ES,50800000
2,Aménagement des consultations externes au rez-de-chaussée du bâtiment principal.,FR,45000000
3,"Limpieza, carboneo y mantenimiento centrales térmicas y ciclos combinados.",ES,76100000
4,"— la location, l'installation et la desinstallation de système embarqué de géolocalisation dans les engins d'exploitation routière,",FR,79714100


In [3]:
df.dropna(inplace=True)

In [4]:
cpv = list(pd.read_csv('cpv/cpv.csv', dtype={'CODE': object})['CODE'])

In [5]:
df['cpv'] = df['cpv'].progress_apply(lambda v: '0' * (8 - len(v)) + v)

  0%|          | 0/2275396 [00:00<?, ?it/s]

In [6]:
def int_to_hierarchy(v):
    labels = [v[:2] + '0' * (len(v) - 2)]
    for i in range(2, len(v)):
        if v[i] == '0':
            break
        labels.append(v[:i + 1] + '0' * (len(v) - i - 1))
    return [l for l in labels if l in cpv]

df['label'] = df.cpv.progress_apply(int_to_hierarchy)

df.head()

  0%|          | 0/2275396 [00:00<?, ?it/s]

,desc,lang,cpv,label
0,Marché de travaux d'entretien en plomberie des parties communes et des logements habites des cites de 13 habitat.,FR,45330000,"[45000000, 45300000, 45330000]"
1,Mantenimiento base de datos de instalaciones (BDI) y Scada.,ES,50800000,"[50000000, 50800000]"
2,Aménagement des consultations externes au rez-de-chaussée du bâtiment principal.,FR,45000000,[45000000]
3,"Limpieza, carboneo y mantenimiento centrales térmicas y ciclos combinados.",ES,76100000,"[76000000, 76100000]"
4,"— la location, l'installation et la desinstallation de système embarqué de géolocalisation dans les engins d'exploitation routière,",FR,79714100,"[79000000, 79700000, 79710000, 79714000, 79714100]"


In [7]:
df.desc = df.desc.str.replace(r'\d{8}(-\d)?', '', regex=True)

In [8]:
from transformers import AutoTokenizer

In [9]:
tokenizer = AutoTokenizer.from_pretrained("EuropeanParliament/EUBERT")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
df['token'] = df.desc.progress_apply(lambda x: tokenizer.tokenize(x))

  0%|          | 0/2275396 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (589 > 512). Running this sequence through the model will result in indexing errors


In [11]:
df.head()

,desc,lang,cpv,label,token
0,Marché de travaux d'entretien en plomberie des parties communes et des logements habites des cites de 13 habitat.,FR,45330000,"[45000000, 45300000, 45330000]","[marche, de, travaux, d, ', entre, ##tien, en, plo, ##mb, ##erie, des, parties, communes, et, des, log, ##ements, hab, ##ites, des, cites, de, 13, habitat, .]"
1,Mantenimiento base de datos de instalaciones (BDI) y Scada.,ES,50800000,"[50000000, 50800000]","[mantenimiento, base, de, datos, de, instalaciones, (, bd, ##i, ), y, scad, ##a, .]"
2,Aménagement des consultations externes au rez-de-chaussée du bâtiment principal.,FR,45000000,[45000000],"[amen, ##agement, des, consultations, externe, ##s, au, rez, -, de, -, chau, ##sse, ##e, du, bati, ##ment, principal, .]"
3,"Limpieza, carboneo y mantenimiento centrales térmicas y ciclos combinados.",ES,76100000,"[76000000, 76100000]","[lim, ##pie, ##za, ,, carbone, ##o, y, mantenimiento, centrales, term, ##icas, y, ciclo, ##s, combina, ##dos, .]"
4,"— la location, l'installation et la desinstallation de système embarqué de géolocalisation dans les engins d'exploitation routière,",FR,79714100,"[79000000, 79700000, 79710000, 79714000, 79714100]","[—, la, location, ,, l, ', installation, et, la, desi, ##ns, ##tal, ##lation, de, systeme, embar, ##que, de, geo, ##loc, ##alisation, dans, les, eng, ##ins, d, ', exploitation, rou, ##tiere, ,]"


In [12]:
import numpy as np

train, validate, test = np.split(df[['label', 'token']].sample(frac=1, random_state=42), [int(.7 * len(df)), int(.85 * len(df))])

/home/ugzuzg/gh/Rooooyy/HiTIN/.venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [13]:
train.to_json('cpv/cpv_train.json', orient='records', lines=True)
validate.to_json('cpv/cpv_val.json', orient='records', lines=True)
test.to_json('cpv/cpv_test.json', orient='records', lines=True)